In [0]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.4.2'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tdqm'
!pip3 install 'eyed3'
!pip3 install 'sox'
!pip3 install 'tflearn'
!pip3 install 'scipy'

ERROR: Could not find a version that satisfies the requirement tdqm (from versions: none)
ERROR: No matching distribution found for tdqm


In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

#import torchvision
#from torchvision import transforms


from PIL import Image
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt



Config


In [0]:
#Spectrogram resolution
pixelPerSecond = 50

#Slice parameters
sliceSize = 224

#Dataset parameters
validationRatio = 0.3
testRatio = 0.1

#Model parameters
batchSize = 128
learningRate = 0.001
nbEpoch = 20

Tools

In [0]:
from IPython.core.debugger import set_trace
#Define Paths
home = 'AIML_project/'
rawDataPath = 'CAL500/songs_selected/'
spectrogramsPath = 'CAL500_test_entire_spectrograms/'
slicesPath = 'CAL500_test_sliced_spectrograms/'
emotions_song_taggedPath = 'CAL500/songs_filtered_with_labels.txt'

#AudioFiles Tools
import eyed3
eyed3.log.setLevel("ERROR") #remove logs

def isMono(filename):
  audiofile = eyed3.load(filename)
  return audiofile.info.mode == 'Mono'

def getEmotion(filename):
  f = open(emotions_song_taggedPath, "r")
  lines = f.readlines()
  emotions = []
  for line in lines:
    if filename in line:
      #set_trace()
      line = line.rstrip(';\n')
      emotions = line.split(':')[1]    
      emotions = emotions.split(',')

      for i in range(len(emotions)):
        emotions[i] = emotions[i].lstrip(' ')

      
  f.close()
  return emotions


#PREPARE DATASET

In [0]:
#from torchvision.datasets import VisionDataset

#from PIL import Image
from IPython.core.debugger import set_trace

import re
import os
import os.path
import sys

# Clone github repository with data
if not os.path.isdir('./AIML_project'):
  !git clone https://github.com/anphetamina/AIML_project.git

sliceSpectgrogram


In [0]:
from PIL import Image
import os.path

#slices all spectrograms
def createSlicesFromSpectrograms(desiredSize):
  if os.path.exists(os.path.dirname(home+slicesPath)):
    os.system("rm -rf AIML_project/CAL500_test_sliced_spectrograms/")
    
  if not os.path.exists(os.path.dirname(home+slicesPath)):
    os.makedirs(os.path.dirname(home+slicesPath))

  files = os.listdir(home+spectrogramsPath)
  files.sort()
  
  for filename in files:
    if filename.endswith(".png"):
      sliceSpectrogram(filename, desiredSize)

#creates slice from spectrogramf
def sliceSpectrogram(filename, desiredSize):
  #emotion = filename.split('-')[2] #TODO get emotion by name
  
 #Load full spectrogram
  img = Image.open(home+spectrogramsPath+filename)

  #compute approximate number of 224x224 samples
  width, height = img.size
  #set_trace()
  nSamples = int(width/desiredSize)
  width-desiredSize


  #for each sample
  for i in range(nSamples):
    print ("Creating slice: '{}/{}' for '{}'".format(i+1, nSamples, filename))
    #extract and save 224x224 sample
    filename = filename.replace('.png','')
    if(i != 0):
      filename = filename.split('_')[0]
      filename = filename+'_{}.png'.format(i+1)
    else:
      filename = filename+'_{}.png'.format(i+1)

    startPixel = i*desiredSize
    imgTmp = img.crop((startPixel, 1, startPixel + desiredSize, desiredSize + 1))
    imgTmp.save(home+slicesPath+filename)
 



#SongToData

In [0]:
from subprocess import Popen, PIPE, STDOUT
import os
from PIL import Image
import eyed3
from IPython.core.debugger import set_trace

#Parameters tunable
desiredSize = 256 #size of png

#Define
currentPath = os.path.dirname(os.path.realpath(home))

#remove logs
eyed3.log.setLevel("ERROR")

def createSpectrogram(filename, newFilename):
  '''
    if isMono(rawDataPath+filename):
      command = "!cp '{}' '/tmp/{}.mp3'".format(rawDataPath+filename, newFilename)
    else:
      command = "!sox '{}' 'tmp/{}/.mp3 remix 1,2".format(rawDataPth+filename, newFilename)
    
    p = Popen(command, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True, cwd=currentPath)
    output, errors = p.communicate()
    if errors:
      print (errors)
  '''
  #CreateSpectrogram
  #filename = filename.replace(".mp3","")

  #set_trace()
  src_path = rawDataPath+filename
  dst_path = spectrogramsPath+newFilename+'.png'
  currentPath = os.getcwd()


  !sox $src_path -n spectrogram -X 56 -x 224 -y 224 -r -o $dst_path
  #command = !'sox {} -n spectrogram -X 56 -x 224 -y 224 -r -o {}'.format(src_path, dst_path)
  #p = Popen(command, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True, cwd=currentPath)
  #output, errors = p.communicate()
  #if errors:
  #  print (errors)
  



#Creates .png whole spectrograms from mp3 files
def createSpectrogramsFromAudio():
   
    EmotionsID_dict = {
      'neutral' : 'e1',
      'calming' : 'e2',
      'happy' : 'e3',
      'sad' : 'e4',
      'angry' : 'e5',
      'fearful' : 'e6'
    }

    songs = os.listdir(rawDataPath)
    songs.sort()
    print(songs)
    print("\n")
    nFiles = len(songs)
    cnt = 0

    #Create path if not existing
    if not os.path.exists(os.path.dirname(spectrogramsPath)):
        os.makedirs(os.path.dirname(spectrogramsPath))



    #Rename files according to emotions
    for index, filename in enumerate(songs):
      #set_trace()
      print ("Creating spectrogram for file '{}' {}:/{}...".format(filename, index+1, nFiles))

      newFilename = str(index+1)
      emotions = getEmotion(filename)
      for emo in emotions:
        emo_label =  EmotionsID_dict['emo']
        newFilename.append("_"+emo_label)

      createSpectrogram(filename, newFilename)
    


#Whole pipeline .mp3 -> .png slices
def createSlicesFromAudio():
  print ("Creating spectrograms...")
  createSpectrogramsFromAudio()
  print ("Spectrograms created.")

  print ("Creating slices...")
  createSlicesFromSpectrograms(desiredSize)
  print ("Slices created.")






In [0]:
!zip -r /content/slices_test.zip /content/AIML_project/CAL500_test_sliced_spectrograms/


  adding: content/AIML_project/CAL500_test_sliced_spectrograms/ (stored 0%)
  adding: content/AIML_project/CAL500_test_sliced_spectrograms/325_32.png (deflated 0%)
  adding: content/AIML_project/CAL500_test_sliced_spectrograms/198_15.png (deflated 0%)
  adding: content/AIML_project/CAL500_test_sliced_spectrograms/214_115.png (deflated 0%)
  adding: content/AIML_project/CAL500_test_sliced_spectrograms/101_43.png (deflated 0%)
  adding: content/AIML_project/CAL500_test_sliced_spectrograms/26_70.png (deflated 0%)
  adding: content/AIML_project/CAL500_test_sliced_spectrograms/340_19.png (deflated 0%)
  adding: content/AIML_project/CAL500_test_sliced_spectrograms/281_34.png (deflated 0%)
  adding: content/AIML_project/CAL500_test_sliced_spectrograms/32_199.png (deflated 0%)
  adding: content/AIML_project/CAL500_test_sliced_spectrograms/189_40.png (deflated 0%)
  adding: content/AIML_project/CAL500_test_sliced_spectrograms/336_15.png (deflated 0%)
  adding: content/AIML_project/CAL500_test_s

FileNotFoundError: ignored

In [0]:
#os.system("rm -rf AIML_project/CAL500_test_sliced_spectrograms/")
files = os.listdir(home+slicesPath)
len(files)

19229

#Other

In [0]:
#createSpectrogramsFromAudio()
createSlicesFromSpectrograms(sliceSize)

Creating slice: '1/65' for '1.png'
Creating slice: '2/65' for '1_1.png'
Creating slice: '3/65' for '1_2.png'
Creating slice: '4/65' for '1_3.png'
Creating slice: '5/65' for '1_4.png'
Creating slice: '6/65' for '1_5.png'
Creating slice: '7/65' for '1_6.png'
Creating slice: '8/65' for '1_7.png'
Creating slice: '9/65' for '1_8.png'
Creating slice: '10/65' for '1_9.png'
Creating slice: '11/65' for '1_10.png'
Creating slice: '12/65' for '1_11.png'
Creating slice: '13/65' for '1_12.png'
Creating slice: '14/65' for '1_13.png'
Creating slice: '15/65' for '1_14.png'
Creating slice: '16/65' for '1_15.png'
Creating slice: '17/65' for '1_16.png'
Creating slice: '18/65' for '1_17.png'
Creating slice: '19/65' for '1_18.png'
Creating slice: '20/65' for '1_19.png'
Creating slice: '21/65' for '1_20.png'
Creating slice: '22/65' for '1_21.png'
Creating slice: '23/65' for '1_22.png'
Creating slice: '24/65' for '1_23.png'
Creating slice: '25/65' for '1_24.png'
Creating slice: '26/65' for '1_25.png'
Creatin

In [0]:
from google.colab import files
files.download("/content/slices_test.zip")